In [ ]:
%reload_ext autoreload
%autoreload 2

from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt
import pandas as pd
from datetime import datetime
from colorama import Fore, Back, Style
import sys
from pprint import pprint
from IPython.display import clear_output
import IPython
from IPython.display import display_html
from btplotting import BacktraderPlottingLive
from btplotting import BacktraderPlotting
import matplotlib 
matplotlib.rc("font", family='Microsoft YaHei')# 增加


sys.path.append("..")
from TestStrategy import TestStrategy
 

from tools.debuger import *

# debug(sys)

cerebro = bt.Cerebro()

cerebro.addstrategy(TestStrategy )

#获取数据
stock_hfq_df = pd.read_csv("./sz000001.csv",index_col='date',parse_dates=True)
start_date = datetime(2019, 8, 1)  # 回测开始时间
end_date = datetime(2022, 8, 30)  # 回测结束时间
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)  # 将数据传入回测系统


cerebro.broker.setcash(100)
# cerebro.addanalyzer(BacktraderPlottingLive)
cerebro.run()
# %matplotlib widget 
# %matplotlib inline 


# output_mode='save', filename='chart.html'
p = BacktraderPlotting(style='bar', output_mode='save', filename='abc.html' )

# ,iplot=False,
print("---show start-------->")
print("--show end--------->")
cerebro.plot(p , iplot=False)
    
# cerebro.plot()
# cerebro.plot( style='bar',iplot=False)



